In [6]:
import os
import numpy as np

import tensorflow as tf
import keras

from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imagenet_utils import decode_predictions
from imagenet_utils import preprocess_input

from tqdm import tqdm
import glob

# Data Prep

### Using ImageDataGenerator.

Eventually want to use args and argparse to take in train_dir and test_dir.

In [2]:
train_dir = 'train_CAL/'
test_dir = 'test_CAL/'

In [3]:
IM_HEIGHT = 224
IM_WIDTH = 224
NB_EPOCHS = 1
BAT_SIZE = 16
FC_SIZE = 500 # May need to train this parameter
nb_classes = 1

In [4]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [7]:
nb_train_samples = get_nb_files(train_dir)
nb_classes = len(glob.glob(train_dir + "/*"))
nb_val_samples = get_nb_files(test_dir)

# VGG Model

In [8]:
def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x)
    predictions = Dense(nb_classes, activation='sigmoid')(x) 
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [12]:
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    for layer in base_model.layers:
        layer.trainable = False
        model.compile(optimizer='adam',    
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])

In [28]:
# ignore this for now
def setup_to_finetune(model):
    """
    If we want to fine-tine
    """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]: # need to declare num layers to train
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=keras.optimizers.Adam(lr=0.0001, momentum=0.9),   
                 loss='binary_crossentropy')

Need to tune:
* architecture type (ResNet, Inception, VGG19)
* FC_SIZE
* Adam parameters (momentum, learning rate, etc)
* image data generator transforming parameters

In [13]:
def vgg_train(train_dir, val_dir):
    
    #train_datagen =  ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rescale=1. / 255)
    train_datagen =  ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)
    
    test_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE
    )
    validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE
    )

    base_model = VGG16(weights='imagenet', include_top=False)
    model = add_new_last_layer(base_model, nb_classes)

    setup_to_transfer_learn(model, base_model)

    history = model.fit_generator(train_generator, epochs = NB_EPOCHS, steps_per_epoch = nb_train_samples, \
                                     validation_data=validation_generator, validation_steps = nb_val_samples)
    model.save("vgg_train.model")
    
    return history

h = vgg_train(train_dir, test_dir)

Found 362 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Epoch 1/1
 12/720 [..............................] - ETA: 4:38:40 - loss: 6.3493 - acc: 0.40 - ETA: 4:15:12 - loss: 7.1603 - acc: 0.45 - ETA: 4:08:23 - loss: 7.4306 - acc: 0.46 - ETA: 3:59:18 - loss: 7.5657 - acc: 0.47 - ETA: 3:53:21 - loss: 7.6468 - acc: 0.48 - ETA: 3:51:09 - loss: 7.7009 - acc: 0.48 - ETA: 3:46:24 - loss: 7.7395 - acc: 0.48 - ETA: 3:45:36 - loss: 7.7685 - acc: 0.48 - ETA: 3:44:26 - loss: 7.7910 - acc: 0.48 - ETA: 3:35:43 - loss: 7.8090 - acc: 0.49 - ETA: 3:36:58 - loss: 7.8238 - acc: 0.49 - ETA: 3:36:37 - loss: 7.8360 - acc: 0.4922

KeyboardInterrupt: 

In [ ]:
def predict(test_dir):
    from keras.models import load_model
    test_datagen = ImageDataGenerator(vertical_flip=True)
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE,
        class_mode="binary"
    )
    
    model = load_model('vgg_train.model')
    pred = model.predict_generator(self, generator, steps, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
    
    return pred

In [70]:
# ignore
#preds = [model.predict(image) for image in tqdm(train_data2)]
#results = [decode_predictions(pred)[0] for pred in preds]
#results